In [ ]:
pip install cdqa
pip install googletrans

In [ ]:
import pandas as pd
from ast import literal_eval

from cdqa.utils.filters import filter_paragraphs
from cdqa.utils.download import download_model, download_bnpp_data
from cdqa.pipeline.cdqa_sklearn import QAPipeline


/usr/local/lib/python3.6/dist-packages/tqdm/autonotebook/__init__.py:18: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [ ]:
from nltk import sent_tokenize, word_tokenize
basedir = '/content/test'
import os
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
files=pd.DataFrame()
files['title'] = os.listdir(basedir)
files['paths']=' '
files['paths'] = [os.path.join(basedir, name) for name in files['title']]
files['paragraphs']=' '
files['paragraphs']=[open((i), 'r', encoding="utf8", errors='ignore').read().strip() for i in files['paths'] ]
files.head()

In [ ]:
files['paragraphs']=files['paragraphs'].apply(lambda x:sent_tokenize(x) )
files.to_csv('new_dataset.csv')

In [ ]:
df=pd.read_csv('new_dataset.csv',converters={'paragraphs': literal_eval})
new_df=df.head(n=3)

In [ ]:
from cdqa.pipeline import QAPipeline
import wget
url = "https://github.com/cdqa-suite/cdQA/releases/download/bert_qa/bert_qa.joblib"
wget.download(url, 'bert_qa.joblib')

'bert_qa (2).joblib'

In [ ]:
cdqa_pipeline = QAPipeline(reader='bert_qa.joblib')
cdqa_pipeline.fit_retriever(df=df)

QAPipeline(reader=BertQA(adam_epsilon=1e-08, bert_model='bert-base-uncased',
                         do_lower_case=True, fp16=False,
                         gradient_accumulation_steps=1, learning_rate=5e-05,
                         local_rank=-1, loss_scale=0, max_answer_length=30,
                         n_best_size=20, no_cuda=False,
                         null_score_diff_threshold=0.0, num_train_epochs=3.0,
                         output_dir=None, predict_batch_size=8, seed=42,
                         server_ip='', server_po...size=8,
                         verbose_logging=False, version_2_with_negative=False,
                         warmup_proportion=0.1, warmup_steps=0),
           retrieve_by_doc=False,
           retriever=BM25Retriever(b=0.75, floor=None, k1=2.0, lowercase=True,
                                   max_df=0.85, min_df=2, ngram_range=(1, 2),
                                   preprocessor=None, stop_words='english',
                                   t

In [ ]:
query = 'In which Academy Viktor Vasnetsov studied ?'
prediction = cdqa_pipeline.predict(query)
prediction

('Abramtsevo',
 'temptext.txt',
 "On the Balcony, Spanish Women Leonora and Ampara, 1897–1898\n\nPolenov introduced Korovin to Savva Mamontov's Abramtsevo Circle: Viktor Vasnetsov, Apollinary Vasnetsov, Ilya Repin, Mark Antokolsky and others.",
 14.170707575123263)

In [ ]:
print('query: {}\n'.format(query))
print('answer: {}\n'.format(prediction[0]))
print('title: {}\n'.format(prediction[1]))
print('paragraph: {}\n'.format(prediction[2]))

query: In which Academy Viktor Vasnetsov studied ?

answer: Abramtsevo

title: temptext.txt

paragraph: On the Balcony, Spanish Women Leonora and Ampara, 1897–1898

Polenov introduced Korovin to Savva Mamontov's Abramtsevo Circle: Viktor Vasnetsov, Apollinary Vasnetsov, Ilya Repin, Mark Antokolsky and others.



In [ ]:
from googletrans import Translator
translator = Translator()
#print(translator.translate('В каком году родился Л?', dest='en',src='ru').text)
query = translator.translate('Кто рисовал в стиле романтизма ?', dest='en',src='ru').text
prediction = cdqa_pipeline.predict(query, n_predictions=5)
prediction
print(translator.translate(prediction[0][0], dest='ru',src='en').text)

Айвазовский


In [ ]:
flag=True
print('Добрый день. Задавайте вопрсы =)')

while(flag==True):
    i=0
    #taking user question
    query = input()
    #query=query.lower()
    
    if('Пока' not in query.split(' ')):
        if(query=='Спасибо' ):
            flag=False
            print("Пожалуйста")
        else:
            query_eng = translator.translate(query, dest='en',src='ru').text
            prediction = cdqa_pipeline.predict(query_eng)
            print("Ответ:"+translator.translate(prediction[0], dest='ru',src='en').text)

                
                
    elif ('Пока' in query.split(' ')):
        flag=False
        

Добрый день. Задавайте вопрсы =)
Кто такой Иван Шишкин?
Ответ:известный пейзажист
Когда родился Врубель?
Ответ:смертный
В каком году родился репин?
Ответ:1876
Какую награду Репин получил за эскиз «Ангел смерти избивает всех перворожденных египтян» 
Ответ:В 1863 году он был награжден двумя серебряными медалями первого класса
Где учился Суриков?
Ответ:Одесская школа рисования
В каком году умер Василий Суриков?
Ответ:1916
Пока


In [ ]:
translator.translate('Какой стиль картин Карл Брюлов ?', dest='en',src='ru').text

'What style of paintings by Karl Bryullov?'